# Import appropriate packages and set analysis options.

In [4]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#plt.style.use('ggplot') 
#from sklearn import linear_model, preprocessing
#from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
#from sklearn.model_selection import cross_val_score, train_test_split
#from sklearn.preprocessing import OneHotEncoder
from IPython.core.interactiveshell import InteractiveShell
#from math import sqrt
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import GradientBoostingRegressor
#from scipy import stats

# Setting save_predicted_data to True
# will cause the notebook to save data
# in one of the last notebook cells.
# The data will be saved to the path
# specified by MY_PREDICTED_DATA_PATH.
save_predicted_data = False

# As expected, setting print_all_output
# to True will cause each evaluation in a
# a cell to be displayed. This has the
# unfortunate side-effect of preventing the
# ';' operator from silencing output.
# If this boolean is set to False, then
# only the last item in each cell may
# be output.
print_all_output = False
InteractiveShell.ast_node_interactivity = 'all' if print_all_output else 'last_expr'

# Setting engineer_features to True will
# enable the creation of new data/features
# from the original data. This makes it
# simpler to include/exclude this extra
# data and determine whether it helps
# improve the models.
# WARNING: if set to True, then these same
# engineered features must be provided/added
# to the test data so that the prediction model
# has the same number of input features for both
# the training and testing data.
engineer_features = False

# Setting one_hot_encoding to True will cause
# categorical features not only to be encoded
# but to be one-hot encoded. This transforms
# all categorical labels into individual
# columns to prevent spurious effects related
# to sequential int encodings.
# WARNING: if set to True, the training and test
# data will be encoded with a different number of
# categorical labels / columns, so the training
# model would not be applicable to the test data.
# Currently, this notebook does not support
# one-hot encoding both the training and testing
# data sets.
one_hot_encoding = False

# Setting randomize_seeding to True will
# randomize various operations throughout
# the notebook. Setting it to False will
# cause the seed to remain fixed to some
# specified value such that the notebook
# can be reran with the same randomized
# variables (see MAGIC_SEED below).
randomize_seeding = False

# Define convenient variables and functions.

In [7]:
# These paths indicate from where the training,
# test, and prediction data will be loaded/saved.
TRAINING_DATA_PATH = "./data/train.csv"
TEST_DATA_PATH = "./data/test.csv"
MY_PREDICTED_DATA_PATH = "./data/my_predicted_survivors.csv"

# The MAGIC_SEED optionally specifies a fixed
# random state/seed so that the notebook can be
# reran with the same randomized variables (see
# randomize_seeding above).
MAGIC_SEED = 1776
if (randomize_seeding):
    MAGIC_SEED = np.random.seed()

# The training data provided with this data
# set will be split into two subsets so that
# models can be trained on the first and tested
# on the second. TRAINING_DATA_TEST_SIZE
# indicates the proportion of the training
# data that will be used as test data for
# model evaluation and should be in the
# range [0.0, 1.0].
TRAINING_DATA_TEST_SIZE = 0.10

def load_data(path, index_column):
    """
    Load the file at 'path' into a Pandas
    DataFrame.
    """
    df = pd.read_csv(path, header=0, index_col=index_column)
    print("Loaded data dimensions: ", df.shape[0], "rows, ", df.shape[1], "columns")
    return df

def print_nan(nan_cols_counts, col_type):
    """
    Print each element of the list which should contain
    a DataFrame feature name and an int number of times
    the feature contains an NaN value.
    """
    print("\n", len(nan_cols_counts), " ", col_type, "-type columns with NaN values.", sep='')
    if(len(nan_cols_counts) > 0):
        print("    {:<16}{}".format("Feature", "NaN Count"))
        print("%s" % "    ---------------------")
    for index, element in enumerate(nan_cols_counts):
        print("{:>2}. {:<16}{}".format(index+1, element[0], element[1]))
        
def gather_nan(df, col_type, print_if_nan = True):
    """
    Find all DataFrame columns of type 'col_type'
    which contain NaN values.
    """
    if (col_type == "int"):
        columns = df.select_dtypes(include=['int']).columns
    elif (col_type == "float"):
        columns = df.select_dtypes(include=['float']).columns
    else:
        columns = df.select_dtypes(include=['object']).columns
    nan_cols_counts = []
    for col in np.sort(columns):
        num_nan = np.sum(df[col].isnull())
        if (num_nan > 0):
            nan_cols_counts.append((col, num_nan))
    if (print_nan):
        print_nan(nan_cols_counts, col_type)
    return nan_cols_counts
            
def replace_with_normal(df, col, seed = np.random.seed()):
    """
    Replace NaN values in a DataFrame column with
    values chosen from a normal distribution with
    a mean and standard deviation equal to the
    that of the non-NaN data.
    """
    np.random.seed(seed)
    df_dropped = df[col].dropna()
    mu = np.mean(df_dropped)
    sigma = np.std(df_dropped)
    null_rows = df[col].isnull()
    num_nan = np.sum(null_rows)
    rand_vals = np.random.normal(mu, sigma, num_nan)
    df.loc[null_rows, col] = rand_vals
    
def most_common_label(df, col):
    """
    Determine the most fequent label for
    categorical data.
    """
    most_common_appearances = 0
    most_common_label = ""
    for label in df[col].unique():
        num_appearances = np.sum(df[col] == label)
        if (num_appearances > most_common_appearances):
            most_common_appearances = num_appearances
            most_common_label = label
    return most_common_label

def evaluate_model(x_train, y_train, model):
    """
    Given an input model and training data,
    split the data into training/testing subsets
    and use this to produce a fit and predictions.
    Indicate the goodness of the fit and plot
    the results.
    """
    # Split the training data into two subsets.
    # Then, train the model on the target data
    # and use it to predict home prices.
    x_train1, x_train2, y_train1, y_train2 = train_test_split(
        x_train, y_train,
        test_size=TRAINING_DATA_TEST_SIZE,
        random_state=MAGIC_SEED)
    model.fit(x_train1, y_train1)
    y_train2_pred = model.predict(x_train2)
    
    # Evaluate the model & predictions by viewing
    # the cross-validation score, error, and
    # variance (where a variance of 1 indicates
    # a perfect prediction) and plotting the results.
    print(cross_val_score(model, x_train1, y_train1, cv=5))
    print("RMS Error: %.3f"
        % sqrt(mean_squared_error(y_train2, y_train2_pred)))
    print('Variance score: %.3f' % r2_score(y_train2, y_train2_pred))
    plt.scatter(y_train2, y_train2_pred, color='red', marker='x')
    plt.scatter(y_train2, y_train2, color='black', marker='.')
    plt.xlabel("True House Price")
    plt.ylabel("House Price")
    plt.legend(("Predicted Value", "True Value"))
    plt.show()
    
def most_important_features(df, feature_importances, num_features):
    """
    Select and pritn out the N most important features used
    in the model to make predictions.
    """
    importances = model.feature_importances_
    sorted_indices = np.argsort(importances)[::-1].tolist()
    top_n_indices = sorted_indices[:5]
    print("The %d most important features for this model:" % num_features)
    for ii, index in enumerate(top_n_indices):
        print("%d. %s" % (ii+1, df.columns[index]))

def encode(df):
    """
    Convert categorical labels to ints.
    """
    label_encoder = preprocessing.LabelEncoder()
    encoded_label_groups = []
    for col in df.select_dtypes(include=['object']).columns:
        label_encoder.fit(df[col].unique());
        encoded_label_groups.append(label_encoder.classes_.tolist())
        df.loc[:, col] = label_encoder.transform(df[col]);
    return encoded_label_groups

# The following function was originally written for debugging
# purposes but was retained in case it was useful in the future.
# For an arbitrary feature index, the function prints details
# about that particular feature's encoding and shows that the
# one_hot_cols columns properly partition/encode the int values
# in the feature's column.
def test_one_hot_encoding(df, features_and_encoded_labels, one_hot_cols, feature_index):
    """
    Print a few elements of an encoded column and compare
    the data to the one-hot encoded columns.
    """
    categorical_features = [pair[0] for pair in features_and_encoded_labels]
    encoded_label_groups = [label for pair in features_and_encoded_labels for label in pair[1]]
    feature = categorical_features[feature_index]
    label_group_starting_index = sum(len(labels) for labels in encoded_label_groups[0:feature_index])
    num_group_labels = len(encoded_label_groups[feature_index])
    feature_label_indices = range(label_group_starting_index, label_group_starting_index+num_group_labels, 1)
    print("# categorical_features: %d\n# encoded_label_groups: %d"
        % (len(categorical_features), len(encoded_label_groups)))
    print("(feature_index, feature, label_group_starting_index, # group labels): (%d, %s, %d, %d)"
        % (feature_index, feature, label_group_starting_index, num_group_labels))
    print(df[feature][0:10])
    for ii in feature_label_indices:
        print(one_hot_cols[ii][0:10])

def one_hot_encode(df, features_and_encoded_labels):
    """
    Perform one-hot encoding on already-encoded data to
    transform each feature label into its own column. This
    helps prevent categorical variable integer mappings
    from indirectly influencing models/fits.
    Assumptions:
    1. The categorical features of the input DataFrame df 
    are already encoded.
    """
    # Gather the already-encoded data.
    categorical_features = [pair[0] for pair in features_and_encoded_labels]
    categorical_values = [df[label].tolist() for label in categorical_features]
    categorical_matrix_transpose = np.array(categorical_values).T.tolist()
    
    # One-hot encode it.
    one_hot_encoder = OneHotEncoder(dtype=int, sparse=False)
    one_hot_encoder.fit(categorical_matrix_transpose)
    print("One-hot encoding %d unique categorical features and %d unique categorical labels"
        % (len(one_hot_encoder.n_values_), sum(one_hot_encoder.n_values_)))
    one_hot_cols = one_hot_encoder.transform(categorical_matrix_transpose).T.tolist()
    
    # Test the encoding.
    if (False):
        dummy_test_index = 0
        test_one_hot_encoding(df, features_and_encoded_labels, one_hot_cols, dummy_test_index)
    
    # Remove the original categorical columns and 
    # add the new one_hot_cols to DataFrame.
    df.drop(categorical_features, axis='columns', inplace=True)
    for index, item in enumerate(features_and_encoded_labels):
        feature = item[0]
        labels = item[1]
        for label in labels:
            df[feature + ":" + label] = one_hot_cols[index]
            
    return one_hot_encoder

# Load and preview the passenger training data.

In [8]:
df = load_data(TRAINING_DATA_PATH, "PassengerId")
df.head()

Loaded data dimensions:  891 rows,  11 columns


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Clean and process the training data.